# Plant Disease Detection Model
### Dataset used : (PlantVillage Dataset)["https://www.kaggle.com/emmarex/plantdisease"]

Required Modules

In [5]:
from keras.models import Sequential
from keras.layers import Convolution2D, Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation, Flatten
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import split_folders

Model Architecture

In [6]:
model = Sequential()

chanDim=-1

model.add(Conv2D(32, (3, 3), padding="same",input_shape=(256,256,3)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(15))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer='adam',metrics=["accuracy"])

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_2 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 85, 85, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_3 (Activation)    (None, 85, 85, 64)      

Splitting data into train and val

In [27]:
split_folders.ratio('dataset/PlantVillage', output="dataset/dataset", seed=1337, ratio=(.8, .2))

Copying files: 20639 files [00:16, 1272.96 files/s]


In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('dataset/dataset/train',
                                                 target_size=(256, 256),
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                subset='training')

test_set = test_datagen.flow_from_directory('dataset/dataset/val',
                                            target_size=(256, 256),
                                            batch_size=32,
                                            class_mode='categorical')


Found 16504 images belonging to 15 classes.
Found 4134 images belonging to 15 classes.


In [ ]:
history=model.fit_generator(
        training_set,
        steps_per_epoch=200, 
        epochs=5,
        validation_data=test_set,
        validation_steps=50)


Epoch 1/5
200/200 [==============================] - 2317s 12s/step - loss: 1.4305 - accuracy: 0.5902 - val_loss: 9.6522 - val_accuracy: 0.1200
Epoch 2/5
200/200 [==============================] - 2280s 11s/step - loss: 0.9392 - accuracy: 0.7166 - val_loss: 8.3032 - val_accuracy: 0.1831
Epoch 3/5
200/200 [==============================] - 2234s 11s/step - loss: 0.7077 - accuracy: 0.7783 - val_loss: 1.3708 - val_accuracy: 0.6188
Epoch 4/5
200/200 [==============================] - 2382s 12s/step - loss: 0.5718 - accuracy: 0.8216 - val_loss: 4.8352 - val_accuracy: 0.4506
Epoch 5/5
 60/200 [========>.....................] - ETA: 26:01 - loss: 0.5163 - accuracy: 0.8318